# MSD Calculator Program
### Migration into Jupyter Notebook Leveraging BokehJS for the Plots

This program is desigend to take csv or tsv data with particle trajectories and calculate mean square displacement for a subset of trajectories that meet the acceptance criteria.

In [12]:
# Importing Libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats 
import math
import tkinter as tk
from tkinter import filedialog as fd


print("import done")

import done


### Packagaes for interactives

In [13]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import widgets
from IPython.display import display, clear_output

print("import done")

import done


In [14]:
# File Reading Function (needs -  files, inputsAnalysis)
def go(minSize, minCount, msdTimeLimit, dt):
    msdRowLimit = math.ceil(msdTimeLimit/dt)
    for fileName in files:
        print("working on file %s" %fileName)
        filePath = fileName
        tmp = fileName.split(' ')
        sampleName = tmp[0]
        df = pd.read_excel(filePath)
        #Filter for entries that have Particle ID = 2, then draw x1 and y1 columns
        #toPlot = df[df['Particle ID'] == 2][[' X1 (pixels)', ' Y1 (pixels)']]
        #plot = toPlot.plot(x=" X1 (pixels)", y=" Y1 (pixels)", kind="scatter")
        #fig = plot.get_figure()
        #fig.savefig("particlePlot.png")
        
        
        #Group Data by id
        grouped = df[[' X1 (pixels)', ' Y1 (pixels)',' X2 (pixels)', ' Y2 (pixels)', ' Particle Size (nm)']].groupby(df['Particle ID'])
        counts = grouped.size() #Creates Series with sizes for each group

        validData = []
        msdData = []
        acceptedParticles = []
        for name, group in grouped:
           
            if counts[name] > float(minCount) and group.iloc[0][' Particle Size (nm)'] > float(minSize):
                #Group passes -> Do things
                #print("Group %s is valid data" %name)
                acceptedParticles.append(name)
                validData.append({'id': "%s" %name, 'data': group})
                r = group[[' X1 (pixels)', ' Y1 (pixels)']]
                
                maxTime = dt*counts[name]
                t = np.linspace(0,maxTime,counts[name])
                traj = pd.DataFrame({'t':t,'x':r[' X1 (pixels)'], 'y':r[' Y1 (pixels)']})
                msds = compute_msd(traj, t_step=dt)
                msdData.append({'id':"%s" %name, 'data':msds})
            #else:
                #Discarded  Data
                #print("Group %s is INVALID data" %name)
        #msdAvgs = pd.DataFrame(columns=['t', 'msdAvg'])
        #for time in np.arange(0, lowestTime, dt):
        times = []
        avgs = []
        #print("The accepted particle id's from file %s are: %s" %(fileName, acceptedParticles))
        #For each time interval, calculate average MSD across all particles
        for index in range(0, msdRowLimit):
            sum = 0.0
            avg = 0.0
            for point in msdData:
                #step = time % .04
                #print(point['data'])
                diff = point['data']['msds'].iloc[index]
                sum = sum + diff
        
            avg = sum/len(msdData)
            time = index * dt
            times.append(time)
            avgs.append(avg)
            
        
        #msdAvgs = pd.Series(data=avgs, index=times)
        msdAvgs = pd.DataFrame(data=avgs, index=times, columns=["Avg MSD"])
        msdAvgs.reset_index(inplace=True)
        msdAvgs.columns = ["Delta Time in Seconds", "Avg MSD"]

        plot(msdAvgs, file)
        
    
    print("Analysis complete")


### MSD calculation function


This function calculates the MSD and takes a dt value
The output of the function is final calculated MSDS value for the given time duration with dt time steps.


In [15]:
def compute_msd(trajectory, t_step, coords=['x', 'y']):
    tau = trajectory['t'].copy()
    shifts = np.floor(tau / t_step).astype(np.int)
    msds = np.zeros(shifts.size)
    msds_std = np.zeros(shifts.size)

    for i, shift in enumerate(shifts):
        diffs = trajectory[coords] - trajectory[coords].shift(-shift)
        sqdist = np.square(diffs).sum(axis=1)
        msds[i] = sqdist.mean()
        msds_std[i] = sqdist.std()

    msds = pd.DataFrame({'msds': msds, 'tau': tau, 'msds_std': msds_std})
    return msds

### bokeh plot generation 

This function creates a bokeh plot from the MSD data

In [16]:
from bokeh.charts import Scatter, output_notebook, show

def plot(df,name):
    p = Scatter(df, x="Delta Time in Seconds", y="Avg MSD", title=name,
            xlabel="dt", ylabel="MSD")

    output_notebook()

    show(p)

### Input File Location Selection
#### Use the file chooser to select the input data folder  

In [17]:
root = tk.Tk()
files = fd.askopenfilenames(parent=root)
root.withdraw()
print("the files chosen were the following")
for file in files:
        print(file + "/n")

the files chosen were the following
E:/Code/pythonPractice/MSD/data/SCNH201_50% 2016-05-17 13-32-34-001-alltracks.xls/n
E:/Code/pythonPractice/MSD/data/SCNH201_50% 2016-05-17 13-27-12-001-alltracks (1).xls/n
E:/Code/pythonPractice/MSD/data/SCNH201_30% 2016-05-17 11-39-31-001-alltracks.xls/n
E:/Code/pythonPractice/MSD/data/file.xls/n


# HERE WE GO

### GET READY TO RUMBLEEEEEEEEEEE!!!!!

The function below runs the code

### Input Analysis Parameters
#### Select the appropriate input parameters in the section below.

The input parameters are as follows
1. minSize
2. minCount
3. msdTimeLimit
4. dt

and stored in a list in the cell below.

In [18]:

minSizeInput = widgets.Text(description="Min Particle Size")
minCountInput = widgets.Text(description="Min Count")
msdTimeLimitInput = widgets.Text(description="Total Analysis Time")
dtInput = widgets.Text(description="delta-T")

display(minSizeInput)
display(minCountInput)
display(msdTimeLimitInput)
display(dtInput)

  
def inputAssignment(b):
    clear_output()
    tmp = True
    for parameter in [minSizeInput, minCountInput, msdTimeLimitInput, dtInput]:
        if parameter.value == "":
            tmp = False
    if tmp:
        print("The chosen analysis parameters are as follows:")
        print(minSizeInput.value, minCountInput.value, msdTimeLimitInput.value, dtInput.value)
        %time go(int(minSizeInput.value), int(minCountInput.value), float(msdTimeLimitInput.value), float(dtInput.value))

    
button  = widgets.Button(description="TIME TO RUMBLE!!")
display(button)

button.on_click(inputAssignment)

    

The chosen analysis parameters are as follows:
100 50 .56 .04
working on file E:/Code/pythonPractice/MSD/data/SCNH201_50% 2016-05-17 13-32-34-001-alltracks.xls


Loading BokehJS ...

working on file E:/Code/pythonPractice/MSD/data/SCNH201_50% 2016-05-17 13-27-12-001-alltracks (1).xls


Loading BokehJS ...

working on file E:/Code/pythonPractice/MSD/data/SCNH201_30% 2016-05-17 11-39-31-001-alltracks.xls


Loading BokehJS ...

working on file E:/Code/pythonPractice/MSD/data/file.xls


Loading BokehJS ...

Analysis complete
Wall time: 1min 15s
